In [ ]:
import numpy               as np
import scipy               as sp
import matplotlib.pyplot   as pp
import pandas              as pd
import seaborn             as sb

from sklearn               import metrics
from sklearn.decomposition import PCA
from sklearn.cluster       import KMeans
from sklearn.mixture       import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from collections           import defaultdict

In [4]:
from utility import html, mark, color, hr, compress

hr('SECTION')
html('<b>hi</b>', [1,2,3], color('hello world', weight = 'bold'), color('!!!', weight = 'bold', background = 'black'))
mark('<b>hi</b>', [1,2,3], color('hello world', weight = 'bold'), color('!!!', weight = 'bold', background = 'black'))
hr()

<span                              ><b>hi</b> [1, 2, 3] <span style="color:red;background:transparent;font-family:monospace;font-weight:bold">hello world</span> <span style="color:red;background:black;font-family:monospace;font-weight:bold">!!!</span></span>

In [5]:
train = pd.read_csv('../input/train.csv', parse_dates = ['first_active_month']).set_index('card_id')
test  = pd.read_csv('../input/test.csv',  parse_dates = ['first_active_month']).set_index('card_id')

In [6]:
merchants = pd.read_csv('../input/merchants.csv').set_index('merchant_id')

In [7]:
transactions_old = pd.read_csv('../input/historical_transactions.csv',   parse_dates = ['purchase_date'])
transactions_new = pd.read_csv('../input/new_merchant_transactions.csv', parse_dates = ['purchase_date'])

In [ ]:
#compress(transactions_old).dtypes

In [11]:
purchase_amount_old = transactions_old.groupby(['card_id'])['purchase_amount'].sum().to_frame('old_purchase_amount') # total old purchase amount per card
purchase_amount_new = transactions_new.groupby(['card_id'])['purchase_amount'].sum().to_frame('new_purchase_amount') # total new purchase amount per card

In [13]:
merchant_visits_old = transactions_old.groupby(['card_id'])['merchant_id'].count().to_frame('old_merchant_visits') # total old merchant visits per card
merchant_visits_new = transactions_new.groupby(['card_id'])['merchant_id'].count().to_frame('new_merchant_visits') # total new merchant visits per card

In [30]:
x_train = train.copy().drop(['target'], axis = 1) # train features
y_train = train.copy().filter(['target'])         # train label
x_train.count()

first_active_month    201917
feature_1             201917
feature_2             201917
feature_3             201917
dtype: int64

In [33]:
x_train = x_train.join(purchase_amount_old[purchase_amount_old.index.isin(train.index)]) # left join old purchase amount on index (card_id), where card_id in train dataset
x_train = x_train.join(purchase_amount_new[purchase_amount_new.index.isin(train.index)]) # left join old purchase amount on index (card_id), where card_id in train dataset

x_train = x_train.join(merchant_visits_old[merchant_visits_old.index.isin(train.index)]) # left join old merchant visits on index (card_id), where card_id in train dataset
x_train = x_train.join(merchant_visits_new[merchant_visits_new.index.isin(train.index)]) # left join old merchant visits on index (card_id), where card_id in train dataset

x_train.count()

first_active_month     201917
feature_1              201917
feature_2              201917
feature_3              201917
old_purchase_amount    201917
new_purchase_amount    179986
old_merchant_visits    201917
new_merchant_visits    179986
dtype: int64

In [35]:
x_train.head()

,first_active_month,feature_1,feature_2,feature_3,old_purchase_amount,new_purchase_amount,old_merchant_visits,new_merchant_visits
card_id,,,,,,,,
C_ID_92a2005557,2017-06-01,5,2,1,-165.968739,-13.244202,254,23.0
C_ID_3d0044924f,2017-01-01,4,1,0,-210.006336,-4.355735,350,6.0
C_ID_d639edf6cd,2016-08-01,2,2,0,-29.167391,-0.700326,43,1.0
C_ID_186d6a6901,2017-09-01,4,3,0,-49.491364,-4.654372,77,7.0
C_ID_cdbd2c0db2,2017-11-01,1,3,0,-48.687656,-19.926237,132,36.0
